<a href="https://colab.research.google.com/github/OSegun/generative-ai/blob/main/Fine_turning_large_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-turning Pegasus-cnn_dailymail Model

The objective of the project is to fine-tine the [pegasus-cnn_dailymail](https://huggingface.co/google/pegasus-cnn_dailymail) model for text summarization on the [Samsung samsum](https://huggingface.co/datasets/Samsung/samsum) dataset from huggingface

In [1]:
!nvidia-smi

Thu Oct 10 16:44:40 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [1]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.8/413.8 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━

In [2]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.9/330.9 kB 14.4 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.34.2
    Uninstalling accelerate-0.34.2:
      Successfully uninstalled accelerate-0.34.2
Found existing installation: transformers 4.44.2
Uninstalling transformers-4.44.2:
  Successfully uninstalled transformers-4.44.2
Found existing installation: accelerate 1.0.0
Uninstalling accelerate-1.0.0:
  Successfully uninstalled accelerate-1.0.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.8 MB/s eta 0:00:00
  Using cached accelerate-1.0.0-py3-none-any.whl.metadata (19 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 98.8 MB/s eta 0:00:00
Using cached accelerate-1.0.0-py3-none-any.whl (330 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 89.6 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully 

In [5]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.7 MB/s eta 0:00:00


In [7]:
import numpy as np
from tqdm import tqdm
import pandas as pd
import torch
from datasets import load_dataset
from transformers import pipeline, set_seed, AutoModelForSeq2SeqLM, AutoTokenizer, DataCollatorForSeq2Seq, TrainingArguments, Trainer
from evaluate import load as load_metric

In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [9]:
google_model = "google/pegasus-cnn_dailymail"
tokenizer = AutoTokenizer.from_pretrained(google_model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

In [10]:
pegasus_model = AutoModelForSeq2SeqLM.from_pretrained(google_model).to(device)

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [11]:
data = load_dataset("samsum")
data

samsum.py:   0%|          | 0.00/3.36k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/7.04k [00:00<?, ?B/s]

The repository for samsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/samsum.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


corpus.7z:   0%|          | 0.00/2.94M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [12]:
data["train"]["dialogue"][2]

"Tim: Hi, what's up?\r\nKim: Bad mood tbh, I was going to do lots of stuff but ended up procrastinating\r\nTim: What did you plan on doing?\r\nKim: Oh you know, uni stuff and unfucking my room\r\nKim: Maybe tomorrow I'll move my ass and do everything\r\nKim: We were going to defrost a fridge so instead of shopping I'll eat some defrosted veggies\r\nTim: For doing stuff I recommend Pomodoro technique where u use breaks for doing chores\r\nTim: It really helps\r\nKim: thanks, maybe I'll do that\r\nTim: I also like using post-its in kaban style"

In [13]:
data["train"][2]["summary"]

'Kim may try the pomodoro technique recommended by Tim to get more stuff done.'

In [14]:
# Preview Data
length_data = [len(data[i]) for i in data]

print("Length of data: ", length_data)
print("Features: ", data["train"].column_names)
print("Dialogue: ", data["test"][1]["dialogue"])
print("Summary: ", data["test"][1]["summary"])

Length of data:  [14732, 819, 818]
Features:  ['id', 'dialogue', 'summary']
Dialogue:  Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I'll check.
Eric: Sure.
Rob: Turns out no! There are some of his stand-ups on youtube.
Eric: Gr8! I'll watch them now!
Rob: Me too!
Eric: MACHINE!
Rob: MACHINE!
Eric: TTYL?
Rob: Sure :)
Summary:  Eric and Rob are going to watch a stand-up on youtube.


In [15]:
# Preprocess data

def features(data):
  input = tokenizer(data["dialogue"], max_length=1024, truncation=True)

  with tokenizer.as_target_tokenizer():
    target = tokenizer(data["summary"], max_length=128, truncation=True)
  return {
      "input_ids" : input['input_ids'],
      'attention_mask' : input['attention_mask'],
      'labels' : target['input_ids']
  }

dataset = data.map(features, batched=True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4109: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [16]:
dataset["train"]

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14732
})

In [17]:
dataset["train"]["input_ids"][2]

[4776,
 151,
 4451,
 108,
 180,
 131,
 116,
 164,
 152,
 5377,
 151,
 6843,
 4301,
 83678,
 108,
 125,
 140,
 313,
 112,
 171,
 1425,
 113,
 1549,
 155,
 2371,
 164,
 64428,
 4776,
 151,
 463,
 368,
 119,
 511,
 124,
 557,
 152,
 5377,
 151,
 4384,
 119,
 235,
 108,
 18857,
 1549,
 111,
 1596,
 2073,
 56616,
 161,
 418,
 5377,
 151,
 3183,
 3469,
 125,
 131,
 267,
 696,
 161,
 130,
 116,
 111,
 171,
 579,
 5377,
 151,
 184,
 195,
 313,
 112,
 38244,
 114,
 5713,
 167,
 1088,
 113,
 1553,
 125,
 131,
 267,
 1461,
 181,
 38244,
 316,
 9609,
 4776,
 151,
 321,
 557,
 1549,
 125,
 1253,
 881,
 93882,
 3111,
 241,
 4911,
 207,
 5033,
 118,
 557,
 18576,
 4776,
 151,
 168,
 288,
 1107,
 5377,
 151,
 1516,
 108,
 1556,
 125,
 131,
 267,
 171,
 120,
 4776,
 151,
 125,
 163,
 172,
 303,
 450,
 121,
 12397,
 115,
 9994,
 11485,
 669,
 1]

In [18]:
dataset["train"]["attention_mask"][2]

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [19]:
dataset["train"]["labels"][2]

[5377,
 218,
 508,
 109,
 891,
 93882,
 3111,
 2087,
 141,
 4776,
 112,
 179,
 154,
 1549,
 479,
 107,
 1]

In [20]:
# Training
#!pip install transformers[torch] -q
#!pip install accelerate==0.26.0
# Loading data in batches in the memory
seq_data = DataCollatorForSeq2Seq(tokenizer, model=pegasus_model)

trainer_args = TrainingArguments(
    output_dir = "peagasus-samsum",
    num_train_epochs=3,
    warmup_steps=500,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=1e6,
    gradient_accumulation_steps=16
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [21]:
trainer = Trainer(
    model = pegasus_model,
    args=trainer_args,
    tokenizer=tokenizer,
    data_collator=seq_data,
    train_dataset=dataset["test"], # Due to the large size of the train data, use the test data
    eval_dataset = dataset["validation"]
)

trainer.train()

Step,Training Loss,Validation Loss


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2618: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=153, training_loss=2.5743486429351607, metrics={'train_runtime': 524.313, 'train_samples_per_second': 4.686, 'train_steps_per_second': 0.292, 'total_flos': 938031889858560.0, 'train_loss': 2.5743486429351607, 'epoch': 2.989010989010989})

In [22]:
# Evaluation

def generate_batch_sized_chunks(list_of_elements, batch_size):
    """Split the dataset into smaller batches that we can process simultaneously."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]

def calculate_metric_on_test_ds(dataset, metric, model, tokenizer, batch_size=16, device=device, column_text="article", column_summary="highlights"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(zip(article_batches, target_batches), total=len(article_batches)):
        inputs = tokenizer(article_batch, max_length=1024, truncation=True, padding="max_length", return_tensors="pt")
        summaries = model.generate(
            input_ids=inputs["input_ids"].to(device),
            attention_mask=inputs["attention_mask"].to(device),
            length_penalty=0.8,
            num_beams=8,
            max_length=128
        )

        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, clean_up_tokenization_spaces=True) for s in summaries]
        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]

        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    score = metric.compute()
    return score


In [25]:
rouge_metric = load_metric('rouge')
# Compute ROUGE scores
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
score = calculate_metric_on_test_ds(
    dataset['test'][0:10],
    rouge_metric,
    trainer.model,
    tokenizer,
    batch_size=2,
    column_text='dialogue',
    column_summary='summary'
)

rouge_dict = dict((rn, score[rn]) for rn in rouge_names)
df = pd.DataFrame(rouge_dict, index=['pegasus'])
df

100%|██████████| 5/5 [00:14<00:00,  2.86s/it]


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.02382,0.0,0.023436,0.023781


In [32]:

## Save model
pegasus_model.save_pretrained("/content/drive/MyDrive/work_data/summarization/pegasus-samsum-model")

## Save tokenizer
tokenizer.save_pretrained("/content/drive/MyDrive/work_data/summarization/tokenizer")

('/content/drive/MyDrive/work_data/summarization/tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/work_data/summarization/tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/work_data/summarization/tokenizer/spiece.model',
 '/content/drive/MyDrive/work_data/summarization/tokenizer/added_tokens.json',
 '/content/drive/MyDrive/work_data/summarization/tokenizer/tokenizer.json')

In [33]:
#Load

tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/work_data/summarization/tokenizer")

In [34]:
#Prediction

gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}



sample_text = dataset["test"][0]["dialogue"]

reference = dataset["test"][0]["summary"]

pipe = pipeline("summarization", model="pegasus-samsum-model",tokenizer=tokenizer)

##
print("Dialogue:")
print(sample_text)


print("\nReference Summary:")
print(reference)


print("\nModel Summary:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Your max_length is set to 128, but your input_length is only 122. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=61)


Dialogue:
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye

Reference Summary:
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.

Model Summary:
Amanda can't find Betty's number. Larry called her last time they were at the park together. Hannah would rather she text him. Amanda Bye bye.
